# Tracer Advection in ... (?pace?)

This notebook demonstrates the computation of 2-dimensional tracer advection implemented in (?pace?). Tracer advection is performed over a single model layer with pre-defined thickness (`pressure_base`), a Gaussian tracer distribution (with maximum value of `tracer_base` that is centered on tile `tracer_target_tile`), and a known wind field.

The mass continuity equation for a conserved scalar mass density $Q$ and horizontal vector velocity $\mathbf{v}$ is

$$
\frac{\partial Q}{\partial t} + \nabla \cdot (Q\mathbf{v}) = 0
$$

We can then use the Divergence theorem to integrate about a quadrilateral grid cell of area $\Delta A$, while simultaneously integrating in time from $t^n$ to $t^{n+1} = t^n + \Delta t$ to express the governing equation in finite (control)-volume form:

$$
\begin{aligned}
\begin{split}
Q^{n+1} & = Q^{n} - \frac{1}{A} \int^{t + \Delta t}_t \oint Q \, \mathbf{v} \cdot \vec{n} \, dl \, dt \\
& = Q^{n} + F[Q, \~u^*] + G[Q, \~v^*] \\
\end{split}
\end{aligned}
$$
where the $F$ and $G$ terms represent time-integrated flux divergences along grid-cell faces in x and y directions.

### Start a parallel cluster

In [1]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines="mpi", n=6).start_and_connect_sync()
%autopx

Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

%autopx enabled


### Input parameters

To set up an FV3 grid on 6 tiles, all you need to define `nx`, `ny` and `nhalo` (these apply to single-tile dimensions). The `timestep` and `nDays` parameters are used in trace advection, where 12 is the number of days it takes for the tracer to propagate around the globe with test case initial conditions.

There are currently two test cases defined:
- <b>a</b>: same configuration as test_case1 in fortran
- <b>b</b>: as in fortran test_case1, but with slightly weaker winds away from the equator, which leads to less dissipation after full rotation.

In [16]:
%%capture
nx = 100
ny = 100
nhalo = 3
""" (int): number of points and halo points in each direction """
timestep = 900.0
""" (int): advection time step in seconds """
nDays = 12
""" (int): number of days to advect for """

test_case = "a"
""" (string): a: fortran test_case1; b: less dispersive """
print_advectionProgress = True
""" (bool): prints out a message every 100th timestep """
plot_outputDuring = False
""" (bool): plot output as advection is active (super slow!) """
plot_outputAfter = False
""" (bool): plot output after advection is complete (slow) """
plot_jupyterAnimation = True
""" (bool): plot an interactive animation after advection is done (slow) """
figure_everyNhours = 6
""" (int): number of model hours between figures """
write_initialCondition = False
""" (bool): creates a netCDF file with initial state """
plot_gridLayout = True
""" (bool): plots and saves grid layout to file grid_map.png """
show_figures = False
""" (bool): shows figures inside Jupyter notebook """


# initial condition adjustables
pressure_base = 10
""" (float): pressure of layer in Pa """
tracer_base = 1.0
""" (float): maximum tracer amount """
tracer_target_tile = 0
""" (int): rank of tile on which to center the tracer (0-5) """

### Configure based on input parameters

In [17]:
%%capture
from mpi4py import MPI
import copy as cp
import functions as func
import os
import importlib
importlib.reload(func)

mpi_comm = MPI.COMM_WORLD

nSeconds = nDays * 86400
figure_everyNsteps = int(3600 / timestep * figure_everyNhours)
nSteps_advection = int(nSeconds / timestep + 1)

if figure_everyNsteps > nSteps_advection:
    figure_everyNsteps = nSteps_advection

pwd = os.getcwd()

write_coordinates = True

namelistDict = func.store_namelist_variables(locals())

plotDict_tracer = {
    "vmin": 0,
    "vmax": tracer_base,
    "units": "",
    "title": "Tracer concentration",
    "cmap": "viridis",
}

### Prepare state for advection

This section steps through everything that is needed in order to run tracer advection. 

1. The domain is configured based on the input `nx`, `ny` and `nhalo` points.
    - The `configuration` dictionary stores all the constructs (I think?) that are needed to initialize stencils.
<br> <br>

2. The initial state fields are created based on chosen test case.
    - The `initialState` dictionary stores `delp`, `uC`, `vC` and `tracer` information required for advection.
<br> <br>

3. The initial state fields are converted into inputs that tracer advection requires.
    - The `tracAdv_data` dictionary stores Courant numbers `crx` and `cry`, mass fluxes `mfxd` and `mfyd`, pressure thickness `delp` and tracer concentration dictionary `tracers`.

In [18]:
metadata, rank = func.define_metadata(namelistDict, mpi_comm)

tracer_dict = {
    "target_tile": tracer_target_tile,
    "rank": rank,
    "tracer_base": tracer_base,
}

configuration = func.configure_domain(mpi_comm, metadata["dimensions"])
if plot_gridLayout:
    func.plot_grid(configuration, metadata, rank, show=show_figures)
if write_coordinates:
    fOut = pwd + "/data/coordinateVariables_%sx%s.nc" % (nx, ny)
    func.write_coordinate_variables_tofile(fOut, metadata, configuration, rank)

initialState = func.create_initial_state(
    configuration["grid_data"], metadata, tracer_dict, pressure_base, test_case
)

tracAdv_data, tracAdv = func.prepare_everything_for_advection(
    configuration, initialState, metadata, timestep
)
tracAdv_dataInit = cp.deepcopy(tracAdv_data)


if write_initialCondition:
    saveDir = pwd + "/data/"
    fOut = saveDir + "initialState_%sx%s_%s.nc" % (nx, ny, test_case)

    func.write_initial_condition_tofile(
        fOut, initialState, metadata, configuration, rank
    )

[stderr:5] /home/ajdas/pace/pace-util/pace/util/grid/gnomonic.py:681: RuntimeWarning: invalid value encountered in true_divide
  np.sum(p * q, axis=-1)


[stderr:2] /home/ajdas/pace/pace-util/pace/util/grid/gnomonic.py:681: RuntimeWarning: invalid value encountered in true_divide
  np.sum(p * q, axis=-1)


[stdout:0] Centering gaussian on lon=354.14, lat=4.13


## Advection loop

This section loops through tracer advection for the specificed number of steps (`nSteps_advection`) that are determined by the requested duration (`nDays`) and given time step (`timestep`). After every time step, the instantaneous tracer state is stored in `tracer_archive`.

You can choose to plot the instantaneous tracer during computation if `plot_outputDuring=True`, but this is very slow. Better plotting options are in the next section.

In [19]:
tracer_archive = [cp.deepcopy(tracAdv_data["tracers"]["tracer"])]
timestep_archive = [0]

for step in range(nSteps_advection):
    if (rank == 0) and (step + 1) % 100 == 0 and print_advectionProgress:
        print("Step: %d" % (step + 1))

    if plot_outputDuring and (step) % figure_everyNsteps == 0:

        tracer_global = configuration["communicator"].gather(tracer_archive[step])

        fOut = (
            pwd
            + "/figs/"
            + "tracerAdvection_%s_%06.2f.png" % (test_case, (step * timestep) / 60 / 60)
        )
        plotDict_tracer["title"] = "Tracer state @ hour: %.2f" % (
            (step * timestep) / 60 / 60
        )
        func.plot_projection_field(
            configuration,
            metadata,
            tracer_global,
            plotDict_tracer,
            rank,
            fOut,
            show=show_figures,
        )

    tracAdv_data = func.run_advection_step_with_reset(
        tracAdv_dataInit, tracAdv_data, tracAdv, timestep
    )

    tracer_archive.append(cp.deepcopy(tracAdv_data["tracers"]["tracer"]))
    timestep_archive.append(step + 1)

print("Done!")


%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

[stdout:0] Step: 100
Step: 200
Step: 300
Step: 400
Step: 500
Step: 600
Step: 700
Step: 800
Step: 900
Step: 1000
Step: 1100
Done!


[stdout:4] Done!


[stdout:5] Done!


[stdout:2] Done!


[stdout:3] Done!


[stdout:1] Done!


### Animation and Plotting

These sections run if `plot_jupyterAnimation` and `plot_outputAfter` are set to `True`.

- `plot_jupyterAnimation` creates an interactive interface where you can click through and play the time progression of tracer advection.
- `plot_outputAfter` plots individual frames into the `/figs/` folder. 

In [20]:
if plot_jupyterAnimation:
    func.plot_tracer_animation(
        configuration,
        metadata,
        tracer_archive,
        rank,
        plotDict_tracer,
        figure_everyNsteps,
        timestep,
    )

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

[output:0]

In [7]:
if plot_outputAfter:
    for step in range(0, nSteps_advection, figure_everyNsteps):
        tracer_global = configuration["communicator"].gather(tracer_archive[step])
        fOut = (
            pwd
            + "/figs/"
            + "tracerAdvection_%s_%06.2f.png" % (test_case, (step * timestep) / 60 / 60)
        )
        plotDict_tracer["title"] = "Tracer state @ hour: %.2f" % (
            (step * timestep) / 60 / 60
        )
        func.plot_projection_field(
            configuration,
            metadata,
            tracer_global,
            plotDict_tracer,
            rank,
            fOut,
            show=show_figures,
        )

In [5]:
gd = configuration['grid_data']

In [7]:
gd.dxa

Out[2:6]: 
CPUStorage([[587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450051, 587436.90345842,
             605885.2928443 , 620037.55883472, 629993.89679152,
             635890.52499425, 637841.36996706, 635890.52499425,
             629993.89679152, 620037.55883472, 605885.2928443 ,
             587436.90345843, 564677.40450051, 537704.11250197,
             537704.11250196, 564677.40450052, 587436.90345843,
                  0.        ],
            [571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
             584539.33649166, 593679.9587589 , 599777.18851088,
             603242.05678331, 604362.48798513, 603242.05678331,
             599777.18851088, 593679.9587589 , 584539.33649166,
             571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
                  0.        ],
            [555541.85135833, 5

Out[3:6]: 
CPUStorage([[587436.90345842, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450052, 587436.90345843,
             605885.2928443 , 620037.55883472, 629993.89679153,
             635890.52499425, 637841.36996707, 635890.52499425,
             629993.89679153, 620037.55883472, 605885.2928443 ,
             587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450051, 587436.90345842,
                  0.        ],
            [571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126426, 555356.72759705, 571902.74906512,
             584539.33649166, 593679.9587589 , 599777.18851088,
             603242.05678331, 604362.48798513, 603242.05678331,
             599777.18851088, 593679.9587589 , 584539.33649165,
             571902.74906513, 555356.72759706, 534589.94126426,
             534589.94126427, 555356.72759706, 571902.74906513,
                  0.        ],
            [555541.85135833, 5

Out[5:6]: 
CPUStorage([[587436.90345842, 564677.40450051, 537704.11250197,
             537704.11250197, 564677.40450052, 587436.90345843,
             605885.2928443 , 620037.55883472, 629993.89679153,
             635890.52499425, 637841.36996707, 635890.52499425,
             629993.89679153, 620037.55883472, 605885.2928443 ,
             587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450052, 587436.90345843,
                  0.        ],
            [571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126426, 555356.72759705, 571902.74906512,
             584539.33649166, 593679.9587589 , 599777.18851088,
             603242.05678331, 604362.48798513, 603242.05678331,
             599777.18851088, 593679.9587589 , 584539.33649166,
             571902.74906513, 555356.72759705, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
                  0.        ],
            [555541.85135833, 5

Out[0:6]: 
CPUStorage([[587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450052, 587436.90345843,
             605885.2928443 , 620037.55883472, 629993.89679153,
             635890.52499425, 637841.36996707, 635890.52499425,
             629993.89679153, 620037.55883472, 605885.2928443 ,
             587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450052, 587436.90345843,
                  0.        ],
            [571902.74906513, 555356.72759705, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
             584539.33649166, 593679.9587589 , 599777.18851088,
             603242.05678331, 604362.48798513, 603242.05678331,
             599777.18851088, 593679.9587589 , 584539.33649166,
             571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
                  0.        ],
            [555541.85135833, 5

Out[4:6]: 
CPUStorage([[587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450051, 587436.90345842,
             605885.2928443 , 620037.55883472, 629993.89679152,
             635890.52499424, 637841.36996707, 635890.52499424,
             629993.89679152, 620037.55883472, 605885.2928443 ,
             587436.90345842, 564677.40450051, 537704.11250196,
             537704.11250197, 564677.40450051, 587436.90345843,
                  0.        ],
            [571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
             584539.33649166, 593679.9587589 , 599777.18851088,
             603242.05678331, 604362.48798513, 603242.05678331,
             599777.18851088, 593679.9587589 , 584539.33649166,
             571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126426, 555356.72759705, 571902.74906513,
                  0.        ],
            [555541.85135833, 5

Out[1:6]: 
CPUStorage([[587436.90345842, 564677.40450051, 537704.11250197,
             537704.11250197, 564677.40450052, 587436.90345843,
             605885.2928443 , 620037.55883472, 629993.89679153,
             635890.52499425, 637841.36996707, 635890.52499425,
             629993.89679153, 620037.55883472, 605885.2928443 ,
             587436.90345843, 564677.40450052, 537704.11250197,
             537704.11250197, 564677.40450051, 587436.90345842,
                  0.        ],
            [571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
             584539.33649166, 593679.9587589 , 599777.18851088,
             603242.05678331, 604362.48798513, 603242.05678331,
             599777.18851088, 593679.9587589 , 584539.33649166,
             571902.74906513, 555356.72759706, 534589.94126427,
             534589.94126427, 555356.72759706, 571902.74906513,
                  0.        ],
            [555541.85135833, 5

## Plot output

In [ ]:
#%autopx

In [ ]:
#cluster.shutdown()